# Loading Kinoeva Data

Loading and parsing text data exported from Kinovea

This is the data that is exported as text format.  For example:

```
#Kinovea Trajectory data export
#T X Y
0:00:00:00 0.00 0.00 
0:00:00:03 1.00 9.00 
0:00:00:06 2.00 20.00 
0:00:00:10 3.00 34.00 
0:00:00:13 5.00 48.00 
0:00:00:16 6.00 66.00 
0:00:00:20 8.00 84.00 
0:00:00:23 8.00 104.00 
0:00:00:26 7.00 125.00 
0:00:00:30 6.00 145.00 
0:00:00:33 7.00 164.00 
0:00:00:36 9.00 182.00 
0:00:00:40 20.00 201.00 

```

In [55]:
# If running from colab
!git clone https://github.com/nicholashojunhui/hss

fatal: destination path 'hss' already exists and is not an empty directory.


In [56]:
import pandas as pd

In [57]:
df = pd.read_csv('/content/hss/data/2/CFF Demo_ Underhand Vertical Med Ball Throw.txt', delim_whitespace=True, index_col=0)
df.head()

,Trajectory,data,export
#Kinovea,,,
#T,X,Y,NaN
0:00:00:00,0.00,0.00,NaN
0:00:00:03,1.00,9.00,NaN
0:00:00:06,2.00,20.00,NaN
0:00:00:10,3.00,34.00,NaN


In [58]:
# drop last column
df.drop(columns=['export'], inplace=True)
df.head()

,Trajectory,data
#Kinovea,,
#T,X,Y
0:00:00:00,0.00,0.00
0:00:00:03,1.00,9.00
0:00:00:06,2.00,20.00
0:00:00:10,3.00,34.00


In [59]:
# replace column names
df.columns = ['X', 'Y']
df.head()

,X,Y
#Kinovea,,
#T,X,Y
0:00:00:00,0.00,0.00
0:00:00:03,1.00,9.00
0:00:00:06,2.00,20.00
0:00:00:10,3.00,34.00


In [60]:
# remove first row
df.drop(['#T'], inplace=True)
df.head()

,X,Y
#Kinovea,,
0:00:00:00,0.00,0.00
0:00:00:03,1.00,9.00
0:00:00:06,2.00,20.00
0:00:00:10,3.00,34.00
0:00:00:13,5.00,48.00


In [61]:
# rename index
df.index.name = 'T'
df.head()

,X,Y
T,,
0:00:00:00,0.00,0.00
0:00:00:03,1.00,9.00
0:00:00:06,2.00,20.00
0:00:00:10,3.00,34.00
0:00:00:13,5.00,48.00


In [62]:
# save the original timestamp as another column
df['Original_timestamp'] = df.index

# time units are in h:mm:ss:hundredth
df.index = pd.to_timedelta(df.index.str.replace(r'(0:.*):(\d+)', r'0\1.\2'), errors='coerce')
df.head()

,X,Y,Original_timestamp
T,,,
0 days 00:00:00,0.00,0.00,0:00:00:00
0 days 00:00:00.030000,1.00,9.00,0:00:00:03
0 days 00:00:00.060000,2.00,20.00,0:00:00:06
0 days 00:00:00.100000,3.00,34.00,0:00:00:10
0 days 00:00:00.130000,5.00,48.00,0:00:00:13


In [63]:
# If you want to convert to dates (non-offset)
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Period.strftime.html
#
# df.index = pd.to_datetime(df.index, format='0:%H:%M:%S', errors='coerce')
# pd.to_datetime(df.index, format='0:%H:%M:%S', errors='coerce')

In [64]:
# convert to numeric
df = df.apply(pd.to_numeric, errors='coerce', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
TimedeltaIndex: 160 entries, 0 days 00:00:00 to 0 days 00:00:05.300000
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   X                   160 non-null    float64
 1   Y                   160 non-null    float64
 2   Original_timestamp  0 non-null      float64
dtypes: float64(3)
memory usage: 5.0 KB


In [65]:
# The above is quite a bit of code, so maybe easier to organize in 
# a function for reusability.  See utils.py

# **Ensure `utils.py` file is within the content folder first before continuing**

In [66]:
from utils import read_kinoeva
df = read_kinoeva('/content/hss/data/2/CFF Demo_ Underhand Vertical Med Ball Throw.txt')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
TimedeltaIndex: 160 entries, 0 days 00:00:00 to 0 days 00:00:05.300000
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   X                   160 non-null    float64
 1   Y                   160 non-null    float64
 2   Original_timestamp  160 non-null    object 
dtypes: float64(2), object(1)
memory usage: 5.0+ KB
None


,X,Y,Original_timestamp
T,,,
0 days 00:00:00,0.0,0.0,0:00:00:00
0 days 00:00:00.030000,1.0,9.0,0:00:00:03
0 days 00:00:00.060000,2.0,20.0,0:00:00:06
0 days 00:00:00.100000,3.0,34.0,0:00:00:10
0 days 00:00:00.130000,5.0,48.0,0:00:00:13
